In [4]:
# imports

%matplotlib inline
import platform
print(platform.python_version())
from ROOT import TFile,vector
import ROOT
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from numpy import mean
from numpy import asarray
from root_pandas import read_root
import math
import json
from math import sqrt,acos,cos,sin,pi,exp,log,isnan,atan2
from textwrap import wrap
import copy

3.5.2


In [5]:
# function to get the new has pi0 flag

def GetNewFlag(truthdata,run2=False,toolarge=False,truthdatapart2=[]):
    # function that will loop through the truth data and look for events to set the newhaspi0 flag
    # requirements: has exactly one pi0 in final state (requirement of statuscode ==1 already set for all events in the jsons)
    
    #set output lists
    run_v = []
    subrun_v = []
    event_v = []
    Enu_true_v = []
    haspi0_v = []
    
    # loop through all events in the truth data
    print(len(truthdata["entries"]))
    for x in range(len(truthdata["entries"])):
        # track progress
        if (x % 50000 == 0):
            print(x)
            
        #check number of pi0 in final state  
        numpi0=0
        for part in range(len(truthdata["entries"][x]['particle_pdg'])):
            pdg = truthdata["entries"][x]['particle_pdg'][part]
            if (pdg == 111):
                #found a pi0!
                numpi0 +=1;
        
        # if exactly one pi0 was found haspi0flag==true, fill all lists
        if numpi0==1:
            run_v.append(truthdata["entries"][x]["run"])
            subrun_v.append(truthdata["entries"][x]["subrun"])
            event_v.append(truthdata["entries"][x]["event"])
            if run2:
                Enu_true_v.append(truthdata["entries"][x]["Enu_true"])
            else:
                Enu_true_v.append(1.0)
            haspi0_v.append(1.0)
            
    if toolarge:
        print("too large, doing part2")
        print(len(truthdatapart2["entries"]))
        for x in range(len(truthdatapart2["entries"])):
            # track progress
            if (x % 50000 == 0):
                print(x)

            #check number of pi0 in final state  
            numpi0=0
            for part in range(len(truthdatapart2["entries"][x]['particle_pdg'])):
                pdg = truthdatapart2["entries"][x]['particle_pdg'][part]
                if (pdg == 111):
                    #found a pi0!
                    numpi0 +=1;

            # if exactly one pi0 was found haspi0flag==true, fill all lists
            if numpi0==1:
                run_v.append(truthdatapart2["entries"][x]["run"])
                subrun_v.append(truthdatapart2["entries"][x]["subrun"])
                event_v.append(truthdatapart2["entries"][x]["event"])
                if run2:
                    Enu_true_v.append(truthdatapart2["entries"][x]["Enu_true"])
                else:
                    Enu_true_v.append(1.0)
                haspi0_v.append(1.0)
             
    return run_v,subrun_v,event_v,Enu_true_v,haspi0_v

In [18]:
# load in the truth variables jsons

f_bnb_run1= open("../data/truthvariables_bnb_overlay_run1.json", "r")
truthdata_bnb_r1 = json.load(f_bnb_run1)
f_bnb_run1_1mil_1= open("../data/truthvariables_numu1_1mil_1.json", "r")
truthdata_bnb_run1_1mil_1 = json.load(f_bnb_run1_1mil_1)
f_bnb_run1_1mil_2= open("../data/truthvariables_numu1_1mil_2.json", "r")
truthdata_bnb_run1_1mil_2 = json.load(f_bnb_run1_1mil_2)
f_bnb_run2= open("../data/truthvariables_bnb_nu_run2.json", "r")
truthdata_bnb_r2 = json.load(f_bnb_run2)
f_bnb_run3= open("../data/truthvariables_bnb_overlay_run3.json", "r")
truthdata_bnb_r3 = json.load(f_bnb_run3)

f_numu_lowE_run1= open("../data/truthvariables_numu1_lowE.json", "r")
truthdata_numu_lowE_r1 = json.load(f_numu_lowE_run1)
f_numu_lowE_run2= open("../data/truthvariables_numu2_lowE.json", "r")
truthdata_numu_lowE_r2 = json.load(f_numu_lowE_run2)
f_numu_lowE_run3= open("../data/truthvariables_numu3_lowE.json", "r")
truthdata_numu_lowE_r3 = json.load(f_numu_lowE_run3)

f_ncpi0_run1 = open("../data/truthvariables_ncpi0_r1.json", "r")
truthdata_ncpi0_r1 =json.load(f_ncpi0_run1)

# setup output list
_bnb_nu_run1_run = []
_bnb_nu_run1_subrun = []
_bnb_nu_run1_event = []
_bnb_nu_run1_Enu_true = []
_bnb_nu_run1_haspi0_new =[]

_bnb_nu_run1_1mil_run = []
_bnb_nu_run1_1mil_subrun = []
_bnb_nu_run1_1mil_event = []
_bnb_nu_run1_1mil_Enu_true = []
_bnb_nu_run1_1mil_haspi0_new =[]


_bnb_nu_run2_run = []
_bnb_nu_run2_subrun = []
_bnb_nu_run2_event = []
_bnb_nu_run2_Enu_true = []
_bnb_nu_run2_haspi0_new =[]

_bnb_nu_run3_run = []
_bnb_nu_run3_subrun = []
_bnb_nu_run3_event = []
_bnb_nu_run3_Enu_true = []
_bnb_nu_run3_haspi0_new =[]

_bnb_nu_run1_lowE_run = []
_bnb_nu_run1_lowE_subrun = []
_bnb_nu_run1_lowE_event = []
_bnb_nu_run1_lowE_Enu_true = []
_bnb_nu_run1_lowE_haspi0_new =[]

_bnb_nu_run2_lowE_lowErun = []
_bnb_nu_run2_lowE_subrun = []
_bnb_nu_run2_lowE_event = []
_bnb_nu_run2_lowE_Enu_true = []
_bnb_nu_run2_lowE_haspi0_new =[]

_bnb_nu_run3_lowE_run = []
_bnb_nu_run3_lowE_subrun = []
_bnb_nu_run3_lowE_event = []
_bnb_nu_run3_lowE_Enu_true = []
_bnb_nu_run3_lowE_haspi0_new =[]



In [23]:
# get trutch info for specific event
# loop through ncpi0 to find truth
print(len(truthdata_ncpi0_r1))
for x in range(len(truthdata_ncpi0_r1["entries"])):
    #find specfic entry
    true_run = truthdata_ncpi0_r1["entries"][x]["run"]
    true_subrun = truthdata_ncpi0_r1["entries"][x]["subrun"]
    true_event = truthdata_ncpi0_r1["entries"][x]["event"]
#     print(true_run)
    if (true_run==7017 and true_subrun==1617 and true_event==80880 ):
        print("here")
        for part in range(len(truthdata_ncpi0_r1["entries"][x]['particle_pdg'])):
            print(truthdata_ncpi0_r1["entries"][x]['particle_pdg'])

1
here
[14, 2112, 111]
[14, 2112, 111]
[14, 2112, 111]


In [7]:
# fill output lists

# _bnb_nu_run1_run,_bnb_nu_run1_subrun,_bnb_nu_run1_event,_bnb_nu_run1_Enu_true,_bnb_nu_run1_haspi0_new = GetNewFlag(truthdata_bnb_r1)
# _bnb_nu_run1_1mil_run,_bnb_nu_run1_1mil_subrun,_bnb_nu_run1_1mil_event,_bnb_nu_run1_1mil_Enu_true,_bnb_nu_run1_1mil_haspi0_new = GetNewFlag(truthdata_bnb_run1_1mil_1,False,True,truthdata_bnb_run1_1mil_2)
# _bnb_nu_run2_run,_bnb_nu_run2_subrun,_bnb_nu_run2_event,_bnb_nu_run2_Enu_true,_bnb_nu_run2_haspi0_new = GetNewFlag(truthdata_bnb_r2,True)
# _bnb_nu_run3_run,_bnb_nu_run3_subrun,_bnb_nu_run3_event,_bnb_nu_run3_Enu_true,_bnb_nu_run3_haspi0_new = GetNewFlag(truthdata_bnb_r3)

_bnb_nu_run1_lowE_run,_bnb_nu_run1_lowE_subrun,_bnb_nu_run1_lowE_event,_bnb_nu_run1_lowE_Enu_true,_bnb_nu_run1_lowE_haspi0_new = GetNewFlag(truthdata_numu_lowE_r1)
_bnb_nu_run2_lowE_run,_bnb_nu_run2_lowE_subrun,_bnb_nu_run2_lowE_event,_bnb_nu_run2_lowE_Enu_true,_bnb_nu_run2_lowE_haspi0_new = GetNewFlag(truthdata_numu_lowE_r2,True)
_bnb_nu_run3_lowE_run,_bnb_nu_run3_lowE_subrun,_bnb_nu_run3_lowE_event,_bnb_nu_run3_lowE_Enu_true,_bnb_nu_run3_lowE_haspi0_new = GetNewFlag(truthdata_numu_lowE_r3)

# print("length run1:",len(_bnb_nu_run1_run))
# print("length run2:",len(_bnb_nu_run2_run))
# print("length run3:",len(_bnb_nu_run3_run))

print("length run1:",len(_bnb_nu_run1_1mil_run))

# save to text files

# file1 = open("2021Jan26_newhaspi0_run1.txt","w")
# # file1.write("[run,subrun,event,newhaspi0]\n")
# for event in range(len(_bnb_nu_run1_run)):
#     teststr = str(_bnb_nu_run1_run[event])+","+str(_bnb_nu_run1_subrun[event])+","+str(_bnb_nu_run1_event[event])+","+str(_bnb_nu_run1_haspi0_new[event])+"\n"
#     file1.write(teststr)
# file1.close()

# file1 = open("2021Feb4_newhaspi0_run1_1mil.txt","w")
# # file1.write("[run,subrun,event,newhaspi0]\n")
# for event in range(len(_bnb_nu_run1_1mil_run)):
#     teststr = str(_bnb_nu_run1_1mil_run[event])+","+str(_bnb_nu_run1_1mil_subrun[event])+","+str(_bnb_nu_run1_1mil_event[event])+","+str(_bnb_nu_run1_1mil_haspi0_new[event])+"\n"
#     file1.write(teststr)
# file1.close()

# file2 = open("2021Jan26_newhaspi0_run2.txt","w")
# # file2.write("[run,subrun,event,Enu_true,newhaspi0]\n")
# for event in range(len(_bnb_nu_run2_run)):
#     teststr = str(_bnb_nu_run2_run[event])+","+str(_bnb_nu_run2_subrun[event])+","+str(_bnb_nu_run2_event[event])+","+str(_bnb_nu_run2_Enu_true[event])+","+str(_bnb_nu_run2_haspi0_new[event])+"\n"
#     file2.write(teststr)
# file2.close()

# file3 = open("2021Jan26_newhaspi0_run3.txt","w")
# # file3.write("[run,subrun,event,newhaspi0]\n")
# for event in range(len(_bnb_nu_run3_run)):
#     teststr = str(_bnb_nu_run3_run[event])+","+str(_bnb_nu_run3_subrun[event])+","+str(_bnb_nu_run3_event[event])+","+str(_bnb_nu_run3_haspi0_new[event])+"\n"
#     file3.write(teststr)
# file3.close()

file1 = open("2021Feb05_newhaspi0_run1_lowE.txt","w")
for event in range(len(_bnb_nu_run1_lowE_run)):
    teststr = str(_bnb_nu_run1_lowE_run[event])+","+str(_bnb_nu_run1_lowE_subrun[event])+","+str(_bnb_nu_run1_lowE_event[event])+","+str(_bnb_nu_run1_lowE_haspi0_new[event])+"\n"
    file1.write(teststr)
file1.close()

file2 = open("2021Feb05_newhaspi0_run2_lowE.txt","w")
for event in range(len(_bnb_nu_run2_lowE_run)):
    teststr = str(_bnb_nu_run2_lowE_run[event])+","+str(_bnb_nu_run2_lowE_subrun[event])+","+str(_bnb_nu_run2_lowE_event[event])+","+str(_bnb_nu_run2_lowE_Enu_true[event])+","+str(_bnb_nu_run2_lowE_haspi0_new[event])+"\n"
    file2.write(teststr)
file2.close()

file3 = open("2021Feb05_newhaspi0_run3_lowE.txt","w")
for event in range(len(_bnb_nu_run3_lowE_run)):
    teststr = str(_bnb_nu_run3_lowE_run[event])+","+str(_bnb_nu_run3_lowE_subrun[event])+","+str(_bnb_nu_run3_lowE_event[event])+","+str(_bnb_nu_run3_lowE_haspi0_new[event])+"\n"
    file3.write(teststr)
file3.close()


40574
0
56435
0
50000
37851
0
length run1: 0
